In [ ]:
import ollama
from lib.models_help.build import get_messages, context_description_image, answer_description_image, get_images
from IPython.display import Markdown, display, clear_output
import re
import tqdm
import lib.utils
import datetime

dict_d = {
    'ciencias-humanas' : 'H',
    'matematica' : 'M',
    'ciencias-natureza' : 'N',
    'linguagens' : 'L'
}

questoes = lib.utils.load_json('./data/questoes/questoes.json')

new_questions = lib.utils.load_json('./data/questoes/new_questions.json')

primary_model = 'gemma2'
secundary_model = 'minicpm-v'

for question in tqdm.tqdm(questoes):
    
    if str(question['id']) in new_questions.keys():
        continue
    
    descriptions = []
    
    images = get_images(question)
    
    if question['type'] in ['context-image', 'full-image']:
        print("Contexto da questão enviados as: ", datetime.datetime.now())
        response = ollama.generate(
            model=secundary_model,
            prompt= context_description_image(question),
            images= [images.pop(0)]
        )
        descriptions.append(response.response)
    
    if question['type'] in ['answer-image', 'full-image']:
        for i, ans in enumerate(["A", "B", "C", "D", "E"], start=1):
            print(f"Alternativa {ans} da questão enviados as: ", datetime.datetime.now())
            ans_prompt_str = answer_description_image(question, ans)
            ans_response = ollama.generate(
                model=secundary_model, 
                prompt=ans_prompt_str,
                images= [images.pop(0)]
            )
            descriptions.append(ans_response.response)
    
    messages = get_messages(question, descriptions, images, 'habilidades')
    

    print("Mensagen enviada as: ", datetime.datetime.now())
    response = ollama.chat(
        model=primary_model,
        messages=messages,
        stream=False,
    )


    response_text = response.message.content

    habilidades = set(re.findall(r'\((H.*?)\)',response_text))
    question['habilidades'] = list(map(lambda x : dict_d.get(question['discipline']) + x,habilidades))
    
    new_questions[str(question['id'])] = question
    
    lib.utils.update_json(new_questions, './data/questoes/new_questions.json')
    clear_output(wait=True)

 29%|██▉       | 729/2529 [1:08:04<4:20:09,  8.67s/it]

Contexto da questão enviados as:  2025-02-27 18:51:57.722865
